In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")

In [3]:
dset_idx <- 3
save_res <- TRUE

In [4]:
gene2id_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/hugo2entrez.tsv")) %>%
    mutate(entrez_gene_id = as.character(entrez_gene_id)) %>%
    rename(entrez = entrez_gene_id)


-- Column specification ------------------------------------------------------------------------------------------------------------
cols(
  hugo_symbol = col_character(),
  entrez_gene_id = col_double()
)




In [5]:
sets <- c("demg", "figo", "survival")
colors <- list(down = "blue", up = "red")

In [6]:
for (set in sets) {
    ea_file <- paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_", set, "_kegg", ".tsv")
    if (file.exists(ea_file)) {
        ea_df <- read_tsv(ea_file)
        dge_res_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv"))

        dir.create(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_", set, "_kegg"), showWarnings = FALSE)
        for (i in seq_len(nrow(ea_df))) {
            pw_name <- ea_df$type[i] %>% str_replace_all(" |-", "_") %>% tolower()
            color_map <- ea_df$geneIDs[i] %>%
                str_split("/") %>%
                unlist() %>%
                as_tibble() %>%
                rename(entrez = value) %>%
                inner_join(gene2id_df, by = "entrez") %>%
                inner_join(dge_res_df, by = c("hugo_symbol" = "geneID")) %>%
                mutate(color = ifelse(log2FoldChange > 0, colors$up, colors$down)) %>%
                select(entrez, color)
            if (save_res) {
                color_map %>% write_tsv(paste0(dirs$analysis_dir, "/enrichment/", unified_dsets[dset_idx], "_", set, "_kegg/", pw_name, ".tsv"))
            }
        }

    }
}


-- Column specification ------------------------------------------------------------------------------------------------------------
cols(
  type = col_character(),
  geneIDs = col_character(),
  count = col_double(),
  ratio = col_double(),
  qval = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------------------
cols(
  type = col_character(),
  geneIDs = col_character(),
  count = col_double(),
  ratio = col_double(),
  qval = col_double()
)



-- Column specification ----------------------------------------------------------------------------------------